In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('preprocessed_data_1.csv')
df

,Name,Body,Year,Price,Transmission,Mileage,Volume,Steering wheel,Customs Clearance,Color,задний,передний,полный
0,5,2,2015,13500000,0,89422.0,2.0,0,1,7,0,0,1
1,4,2,2012,10000000,3,44000.0,2.0,0,1,11,0,0,1
2,3,9,2018,41500000,0,142000.0,3.0,0,1,11,0,0,1
3,1,5,2011,8399000,3,193250.0,2.0,0,1,8,0,0,1
4,0,12,2012,3800000,3,140000.0,1.2,0,0,9,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6614,238,0,2015,40000000,0,80000.0,5.0,0,1,8,0,0,1
6615,239,2,2016,16000000,0,60000.0,2.0,0,1,1,0,0,1
6616,239,2,2014,12900000,0,94900.0,2.0,0,1,4,0,0,1
6617,239,2,2013,11500000,0,81000.0,2.0,0,1,1,0,1,0


In [3]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.cross_decomposition import PLSRegression

In [4]:
# Separating out the features and target variable
X = df.drop('Price', axis=1)  # Assuming 'Price' is the target variable
y = df['Price']

# Standardizing the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Applying PCA
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [6]:
max_acc=0
ind=0
for n in range(1,50):
    knn = KNeighborsRegressor(n_neighbors=n)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = r2_score(y_test, y_pred)  # R-squared as a measure of accuracy
    if accuracy>max_acc:
        max_acc=accuracy
        ind=n
print(ind, max_acc)

13 0.8033619186774112


In [7]:
parameters = {
    'kernel': ['linear', 'poly', 'rbf'],
    'C': [1, 10, 100],
    'epsilon': [0.1, 0.5, 1]
}

svr = SVR()
grid_search = GridSearchCV(svr, parameters, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best parameters and best score
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_parameters)
print("Best Score:", best_score)

Best Parameters: {'C': 100, 'epsilon': 0.1, 'kernel': 'linear'}
Best Score: 0.01182094046777602


In [8]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid to search
param_grid = {
    'depth': [6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [30, 50, 100]
}

# Create a base model
catboost = CatBoostRegressor()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=catboost, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Best parameters
grid_search.best_params_


Fitting 3 folds for each of 27 candidates, totalling 81 fits
0:	learn: 7922648.6010566	total: 156ms	remaining: 15.4s
1:	learn: 7365459.0715446	total: 166ms	remaining: 8.14s
2:	learn: 6863568.9161517	total: 175ms	remaining: 5.67s
3:	learn: 6438326.5225385	total: 185ms	remaining: 4.44s
4:	learn: 6067913.6474005	total: 194ms	remaining: 3.69s
5:	learn: 5695888.2281079	total: 204ms	remaining: 3.19s
6:	learn: 5414336.9453364	total: 213ms	remaining: 2.83s
7:	learn: 5101252.3988760	total: 223ms	remaining: 2.56s
8:	learn: 4846176.7733225	total: 232ms	remaining: 2.34s
9:	learn: 4613642.5115516	total: 241ms	remaining: 2.17s
10:	learn: 4415270.8924742	total: 250ms	remaining: 2.02s
11:	learn: 4246371.6390291	total: 259ms	remaining: 1.9s
12:	learn: 4078044.6208887	total: 268ms	remaining: 1.79s
13:	learn: 3941680.6730172	total: 277ms	remaining: 1.7s
14:	learn: 3800291.0273157	total: 286ms	remaining: 1.62s
15:	learn: 3658595.6334210	total: 297ms	remaining: 1.56s
16:	learn: 3549916.1840918	total: 310ms

{'depth': 8, 'iterations': 100, 'learning_rate': 0.1}

In [6]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Support Vector Regressor': SVR(C=100, epsilon = 0.1, kernel = 'linear'),
    'K-Nearest Neighbors': KNeighborsRegressor(n_neighbors=13),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'AdaBoost': AdaBoostRegressor(random_state=42),
    'Bagging': BaggingRegressor(random_state=42),
    'Ridge Regression': Ridge(random_state=42),
    'Lasso Regression': Lasso(random_state=42),
    'ElasticNet': ElasticNet(random_state=42),
    'XGBoost': XGBRegressor(random_state=42),
    'LightGBM': LGBMRegressor(random_state=42),
    'CatBoost': CatBoostRegressor(random_state=42, verbose=0),
    'PLSRegression': PLSRegression(n_components = 12),
    'MLPRegressor': MLPRegressor(hidden_layer_sizes = 50, alpha = 0.001, solver = 'lbfgs', learning_rate = 'adaptive'),
    'ExtraTreesRegressor': ExtraTreesRegressor(),

}

# Training and evaluating models
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = r2_score(y_test, y_pred)  # R-squared as a measure of accuracy
    results[name] = accuracy

results

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000274 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 517
[LightGBM] [Info] Number of data points in the train set: 5295, number of used features: 12
[LightGBM] [Info] Start training from score 9080481.488196


C:\Users\Damir\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


{'Linear Regression': 0.5540396188146894,
 'Random Forest': 0.8949464748778169,
 'Support Vector Regressor': 0.058900323465795434,
 'K-Nearest Neighbors': 0.8033619186774112,
 'Decision Tree': 0.8301360967700739,
 'Gradient Boosting': 0.8669955323743723,
 'AdaBoost': 0.284308825632554,
 'Bagging': 0.8798164531608794,
 'Ridge Regression': 0.5540610618019511,
 'Lasso Regression': 0.5540112798961466,
 'ElasticNet': 0.5479896779151112,
 'XGBoost': 0.9040174472505607,
 'LightGBM': 0.9078528004477533,
 'CatBoost': 0.9242536944671232,
 'PLSRegression': -64.28745050825692,
 'MLPRegressor': 0.6976598265260776,
 'ExtraTreesRegressor': 0.9119551807897307}

In [10]:
import statsmodels.api as sm

#add constant to predictor variables
X_sm = sm.add_constant(X)

#fit linear regression model
model = sm.OLS(y, X_sm).fit()

#view model summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.540
Model:                            OLS   Adj. R-squared:                  0.539
Method:                 Least Squares   F-statistic:                     705.4
Date:                Tue, 12 Dec 2023   Prob (F-statistic):               0.00
Time:                        16:39:55   Log-Likelihood:            -1.1220e+05
No. Observations:                6619   AIC:                         2.244e+05
Df Residuals:                    6607   BIC:                         2.245e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const             -1.261e+09   3.08e+0

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the neural network architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='linear')  # For regression
])

# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Fit the model to the training data
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model on the test data
y_pred = model.predict(X_test)
accuracy = r2_score(y_test, y_pred)
print('R-squared for Neural Network:', accuracy)




Epoch 1/100

133/133 [==============================] - 2s 4ms/step - loss: 159156338163712.0000 - val_loss: 140567426105344.0000
Epoch 2/100
133/133 [==============================] - 0s 3ms/step - loss: 159144409563136.0000 - val_loss: 140542260281344.0000
Epoch 3/100
133/133 [==============================] - 0s 3ms/step - loss: 159094866444288.0000 - val_loss: 140466485985280.0000
Epoch 4/100
133/133 [==============================] - 0s 3ms/step - loss: 158977895694336.0000 - val_loss: 140312596971520.0000
Epoch 5/100
133/133 [==============================] - 1s 5ms/step - loss: 158772190248960.0000 - val_loss: 140062683561984.0000
Epoch 6/100
133/133 [==============================] - 1s 4ms/step - loss: 158460066922496.0000 - val_loss: 139699070959616.0000
Epoch 7/100
133/133 [==============================] - 0s 3ms/step - loss: 158022215139328.0000 - val_loss: 139200905084928.0000
Epoch 8/100
133/133 [==============================] - 1s 4ms/step - loss: 157424342269952.000